# аналитика тираэт

In [25]:
import pandas as pd
from tqdm import trange, tqdm
import numpy as np
from IPython.display import display
import requests
from bs4 import BeautifulSoup as bs
import re

In [2]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', 1000)

In [55]:
df = pd.read_csv('csv/tiraet_5_6_2024.csv')

In [4]:
brend = pd.read_excel('csv/brend.xlsx')

In [5]:
def lower(x):
    return x.lower()

In [6]:
brend['names'] = brend['name'].apply(lower)

In [7]:
brend_up = brend.name.unique()
brend_low = brend.names.unique()

загружаем и создаем переменную с брендами

In [8]:
def brends(x):
    xx = x.replace('\xa0', ' ').replace(',', ' ')
    xx = xx.split(' ')    
    for i in range(len(xx)-1):
        if xx[i].lower() in brend_low:
            return xx[i].lower()
        elif (str(xx[i].lower())+' '+str(xx[i+1].lower())) in brend_low:
            return (str(xx[i].lower())+' '+str(xx[i+1].lower()))    
    else:    
        return 'другой'

In [9]:
def category(x):
    xx = x.split('/')
    xx = (xx[4].replace('Komplektujuschie','КОМПЬЮТЕРНАЯ_ТЕХНИКА').
          replace('2873','МОБИЛЬНЫЕ_ТЕЛЕФОНЫ').
          replace('Kompjuternaja-mebel','ДЛЯ_ДОМА').
          replace('bytovaya','БЫТОВАЯ_ТЕХНИКА').
          replace('ohrannye-sistemy','БЕЗОПАСНОСТЬ').
          replace('svet-i-jelektrika','СВЕТОТЕХНИКА').
          replace('orgtehnika','ОРГТЕХНИКА').
          replace('PHoto--i-video','ФОТО_АУДИО').
          replace('setevoe-oborudovanie','LAN_ОБОРУДОВАНИЕ').
          replace('jelektropitanie','ЭЛЕКТРОПИТАНИЕ').
          replace('uslugi','СПОРТ').
          replace('avtojelektronika','АВТОТОВАРЫ').
          replace('rashodnye-materialy','РАСХОДНИКИ'))
    return xx    

In [68]:
def subcategory(x):
    reqx = requests.get(x)
    soup = bs(reqx.text, 'html.parser')    
    data = []
    data.append(soup.findAll('span', class_='breadcrumbs__item-name font_xs')[3].text)

                    
    return data[0]

In [11]:
def group(x):
    xx = x.split('/')
    if len(xx) == 7:
        return xx[-3].replace('-','_')
    elif len(xx) == 8:
        return xx[-3].replace('-','_')
    elif len(xx) == 9:
        return xx[-4].replace('-','_')
    elif len(xx) == 10:
        return xx[-5].replace('-','_')
    elif len(xx) == 11:
        return xx[-6].replace('-','_')  

In [12]:
df['action_sale'] = 0

In [13]:
df.loc[df['old_price'] > 0,'action_sale'] = 1

создаем столбец который показывает является ли товар акционным(1 - акционный, 0 - не акционный)

In [14]:
df['sale'] = (100 - ((df['price']*100)/df['old_price'])).round(2) 

создаем столбец показывающий скидку в процентах

In [15]:
df['category'] = df['urls'].apply(category)

Создаем столбец показывающий категорию товаров

In [70]:
df['subcategory'] = df['urls'].apply(subcategory_new)

создаем столбец показывающий подкатегорию товаров

In [83]:
reqx = requests.get('https://tiraet.com/catalog/bytovaya/melkaja-bytovaja-tehnika/mashinki-dlja-udalenija-katyshkov/')
soup = bs(reqx.text, 'html.parser')   

In [84]:
try:
    x = soup.findAll('span', class_='breadcrumbs__item-name font_xs')[4].text
except:
    x = soup.findAll('span', class_='breadcrumbs__item-name font_xs')[3].text
    

In [85]:
x

'Машинки для удаления катышков'

In [73]:
df = df.head(5)

In [74]:
df

,title,urls,datе,counts,id,price,old_price,subcategory_new
11419,Головка для плоттеров Canon PF-03 (2251B001),https://tiraet.com/catalog/rashodnye-materialy/zapchasti-dlja-printerov-i-kopirov/zapchasti_print/golovka-dlja-plotterov-canon-pf-03-2251b001/,2024-06-05,Мало,27477,7920.0,NaN,"ЗАПЧАСТИ ПРИНТЕРОВ, КОПИРОВ, ФАКСОВ"
11420,Узел термозакрепления (для сетевых апп)CANON iR2202/2204(FM1-F162-0XX),https://tiraet.com/catalog/rashodnye-materialy/zapchasti-dlja-printerov-i-kopirov/zapchasti_print/uzel_termozakrepleniya_dlya_setevykh_app_canon_ir2202_2204_fm1_f162_0xx/,2024-06-05,Мало,55309,8085.0,NaN,"ЗАПЧАСТИ ПРИНТЕРОВ, КОПИРОВ, ФАКСОВ"
11421,Головка для плоттеров Canon PF-04 (3630B001),https://tiraet.com/catalog/rashodnye-materialy/zapchasti-dlja-printerov-i-kopirov/zapchasti_print/golovka_dlya_plotterov_canon_pf_04_3630b001/,2024-06-05,Мало,56144,8250.0,NaN,"ЗАПЧАСТИ ПРИНТЕРОВ, КОПИРОВ, ФАКСОВ"
11422,Картридж EPSON MX-80/LX-300/400/800/850/880 Patron,https://tiraet.com/catalog/rashodnye-materialy/Rashodnye-k-matrichnym-printeram-kartridzhi-lenty/kartridji_matr/kartridzh-epson-lx-300400800850880mx-80/,2024-06-05,Нет в наличии,249,38.0,NaN,РАСХОДНЫЕ МАТРИЧНЫХ ПРИНТЕРОВ
11423,"Комплект восстановления Canon IR-1435 CET5280N (фотобарабан, ракель, ВПЗ,чип ф/б)",https://tiraet.com/catalog/rashodnye-materialy/Rashodnye-k-lazernym-printeram-kartridzhi-toner/chip_laser/komplekt_vosstanovleniya_canon_ir_1435_cet5280n_fotobaraban_rakel_vpz_chip_f_b/,2024-06-05,Нет в наличии,68689,594.0,NaN,"РАСХОДНЫЕ КАРТРИДЖЕЙ ЛАЗЕРНЫХ ПРИНТЕРОВ, ФАКСОВ"


In [17]:
df['brend'] = df['title'].apply(brends)